In [1]:
import pandas as pd
import pyodbc

# Initialize connection parameters

In [2]:
DB = {'servername': 'MSI',
      'source': 'AdventureWorks2019',
      'destination':'DW',
      'user':'user1',
      'pass':'123456'}

# Create connection

In [3]:
#SourceDB
sourceConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['source'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')

#DestinationDB
destConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE='+DB['destination'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')
cursor=destConn.cursor()

In [4]:
# df=pd.read_sql('SELECT * FROM Purchasing.ShipMethod',sourceConn,index_col="ShipMethodID")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
# df = pd.read_sql('SELECT * FROM Purchasing.[PurchaseOrderHeader]',sourceConn)
# df['ModifiedDate'] =pd.to_datetime(df['ModifiedDate']).dt.date
# date=df['ModifiedDate'].drop_duplicates().reset_index(drop=True)
# print(date)

# Extract data from source DB to pandas DataFrames

In [4]:
Tables= [('Purchasing.ShipMethod','ShipMethodID'),('Production.Product','ProductID'),('Purchasing.Vendor','BusinessEntityID'),('Purchasing.PurchaseOrderHeader','PurchaseOrderID'),
         ('Purchasing.PurchaseOrderDetail',['PurchaseOrderID','PurchaseOrderDetailID']), 
         ('Person.Person','BusinessEntityID'),('Sales.SalesTerritory','TerritoryID'),
         ('Person.CountryRegion','CountryRegionCode'),('Sales.SalesOrderHeader','SalesOrderID'),('Sales.SalesOrderDetail',['SalesOrderID','SalesOrderDetailID'])
        ]

In [5]:
# df=pd.read_sql('SELECT BusinessEntityID FROM HumanResources.Employee',sourceConn,index_col='BusinessEntityID')
# df


In [6]:
dfs={}
for item in Tables:
    dfs[item[0]]=pd.read_sql(f'SELECT * FROM {item[0]}',sourceConn,index_col=item[1])
dfs['HumanResources.Employee']=pd.read_sql('SELECT BusinessEntityID FROM HumanResources.Employee',sourceConn,index_col='BusinessEntityID')


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas

# Transform staged data

## Extract essential columns for data warehouse

In [7]:
#columns to extract in each table
Columns= [('Purchasing.ShipMethod','Name'),('Production.Product',['Name','ProductLine']),('Purchasing.Vendor',['Name','CreditRating']),
         ('Purchasing.PurchaseOrderHeader',['Status','EmployeeID','VendorID','ShipMethodID','OrderDate','ShipDate']),
         ('Purchasing.PurchaseOrderDetail',['DueDate','OrderQty','ProductID','LineTotal','ReceivedQty','RejectedQty','StockedQty']), 
         ('Person.Person',['FirstName','MiddleName','LastName','Suffix']),('Sales.SalesTerritory',['Name','CountryRegionCode','Group']),
         ('Person.CountryRegion','Name'),
         ('Sales.SalesOrderHeader',['OrderDate','DueDate','ShipDate','Status','OnlineOrderFlag','TerritoryID','ShipMethodID']),
         ('Sales.SalesOrderDetail',['OrderQty','ProductID','LineTotal'])
        ]

In [ ]:
# df=pd.DataFrame(dfs['Production.Product'][['Name','Color','Size','Class','Style']])
# df

In [15]:
dfst={}
for item in Columns:
    dfst[item[0]]=dfs[item[0]][item[1]]
dfst['HumanResources.Employee']=dfs['HumanResources.Employee']

## Transform  Data for Dimensions

### DimEmpolyee

In [ ]:
dfst['Person.Person']['Name']=dfst['Person.Person']['FirstName'].map(str)+' '+dfst['Person.Person']['MiddleName'].map(str)+'. '+dfst['Person.Person']['LastName'].map(str)
dfst['HumanResources.Employee']=pd.merge(dfst['HumanResources.Employee'],dfst['Person.Person'],how='left',on='BusinessEntityID')

### DimProduct

In [22]:
dfst['Production.Product']

,Name,ProductLine
ProductID,,
1,Adjustable Race,Accessory
2,Bearing Ball,Accessory
3,BB Ball Bearing,Accessory
4,Headset Ball Bearings,Accessory
316,Blade,Accessory
...,...,...
995,ML Bottom Bracket,Accessory
996,HL Bottom Bracket,Accessory
997,"Road-750 Black, 44",Road


In [17]:
mapper={None:'Accessory','R':'Road','M':'Mountain','T':'Touring','S':'Standard'}
dfst['Production.Product']['ProductLine']=dfst['Production.Product']['ProductLine'].replace(mapper,regex=True)
dfst['Production.Product']

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9000\368933943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfst['Production.Product'].loc[:,'ProductLine']=dfst['Production.Product'].loc[:,'ProductLine'].replace(mapper,regex=True)


ProductID
1      Accessory
2      Accessory
3      Accessory
4      Accessory
316    Accessory
         ...    
995    Accessory
996    Accessory
997        Road 
998        Road 
999        Road 
Name: ProductLine, Length: 504, dtype: object

### DimDate

In [76]:
df=pd.concat([dfst['Purchasing.PurchaseOrderHeader']['OrderDate'],dfst['Purchasing.PurchaseOrderDetail']['DueDate'],dfst['Purchasing.PurchaseOrderHeader']['ShipDate'],
dfst['Sales.SalesOrderHeader']['OrderDate'],dfst['Sales.SalesOrderHeader']['DueDate'],dfst['Sales.SalesOrderHeader']['ShipDate']],ignore_index=True,axis=0)

In [77]:
df=df.drop_duplicates().sort_values()
df=pd.DataFrame({'FullDate':df})
df

,FullDate
0,2011-04-16
12857,2011-04-25
4,2011-04-30
12861,2011-05-09
4017,2011-05-14
...,...
12733,2014-08-16
12763,2014-08-17
4001,2014-09-22
16858,2014-10-17


In [78]:
df['DayOfWeek']=df['FullDate'].dt.day_name()
df


,FullDate,DayOfWeek
0,2011-04-16,Saturday
12857,2011-04-25,Monday
4,2011-04-30,Saturday
12861,2011-05-09,Monday
4017,2011-05-14,Saturday
...,...,...
12733,2014-08-16,Saturday
12763,2014-08-17,Sunday
4001,2014-09-22,Monday
16858,2014-10-17,Friday


### DimShipMethod

In [57]:
dfst['Purchasing.ShipMethod']=pd.DataFrame(dfst['Purchasing.ShipMethod'])

In [58]:
dfst['Purchasing.ShipMethod']

,Name
ShipMethodID,
1,XRQ - TRUCK GROUND
2,ZY - EXPRESS
3,OVERSEAS - DELUXE
4,OVERNIGHT J-FAST
5,CARGO TRANSPORT 5


### DimVendor

In [54]:
dfst['Purchasing.Vendor']

,Name,CreditRating
BusinessEntityID,,
1492,Australia Bike Retailer,1
1494,Allenson Cycles,2
1496,Advanced Bicycles,1
1498,"Trikes, Inc.",2
1500,Morgan Bike Accessories,1
...,...,...
1690,Bloomington Multisport,1
1692,Carlson Specialties,2
1694,"Compete, Inc.",1


In [61]:
dfst['Purchasing.Vendor']['CreditRating']=dfst['Purchasing.Vendor']['CreditRating'].map({1:'Superior',2:'Excellent',3:'Above Average',4:'Average',5:'Below Average'})

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9000\229811126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfst['Purchasing.Vendor']['CreditRating']=dfst['Purchasing.Vendor']['CreditRating'].map({1:'Superior',2:'Excellent',3:'Above Average',4:'Average',5:'Below Average'})


In [62]:
dfst['Purchasing.Vendor']

,Name,CreditRating
BusinessEntityID,,
1492,Australia Bike Retailer,Superior
1494,Allenson Cycles,Excellent
1496,Advanced Bicycles,Superior
1498,"Trikes, Inc.",Excellent
1500,Morgan Bike Accessories,Superior
...,...,...
1690,Bloomington Multisport,Superior
1692,Carlson Specialties,Excellent
1694,"Compete, Inc.",Superior


### DimTerritory

In [95]:
pd.merge(dfst['Sales.SalesTerritory'],dfst['Person.CountryRegion'],on='CountryRegionCode').rename(columns={'Name_y':'CountryRegion'})

,Name_x,CountryRegionCode,Group,CountryRegion
0,Northwest,US,North America,United States
1,Northeast,US,North America,United States
2,Central,US,North America,United States
3,Southwest,US,North America,United States
4,Southeast,US,North America,United States
5,Canada,CA,North America,Canada
6,France,FR,Europe,France
7,Germany,DE,Europe,Germany
8,Australia,AU,Pacific,Australia
9,United Kingdom,GB,Europe,United Kingdom


### DimPurchasingStatus

In [84]:
df1=pd.DataFrame({'StatusID':[1,2,3,4],'Name':['Pending','Approved','Rejected','Completed']})
df1

,StatusID,Name
0,1,Pending
1,2,Approved
2,3,Rejected
3,4,Completed


### DimSalesStatus

In [85]:
df2=pd.DataFrame({'StatusID':[1,2,3,4,5,6],'Name':['In process','Approved','Backordered','Rejected','Shipped','Cancelled']})
df2

,StatusID,Name
0,1,In process
1,2,Approved
2,3,Backordered
3,4,Rejected
4,5,Shipped
5,6,Cancelled


### DimOnlineOrderFlag

In [87]:
df3=pd.DataFrame({'StatusID':[0,1],'Name':['Sale person','Online']})
df3

,StatusID,Name
0,0,Sale person
1,1,Online


## Transform data for Fact

# Load to Data Warehouse

In [24]:
for index, row in df.iterrows():
      cursor.execute('''INSERT INTO dbo.DimEmployee (EmployeeID,Name) values(?,?)''',index, row.Name)      
cursor.commit()
